In [1]:
import requests
import pandas as pd
import json

In [2]:
cols = 'adult, genre_ids, id, original_language, original_title, release_date, vote_average, vote_count'.split(', ')
df = pd.DataFrame(columns=cols)
for page in range(1,11):
    url = f"https://api.themoviedb.org/3/movie/popular?api_key=552dbe069b589a399d500b9bdfcbf6e7&language=en-US&page={page}"
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    jdata = json.loads(response.text)
    df_temp = pd.DataFrame(jdata['results'])[cols]
    df = df.append(df_temp)


C:\Users\conta\AppData\Local\Temp\ipykernel_20848\3932318407.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp)
C:\Users\conta\AppData\Local\Temp\ipykernel_20848\3932318407.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp)
C:\Users\conta\AppData\Local\Temp\ipykernel_20848\3932318407.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp)
C:\Users\conta\AppData\Local\Temp\ipykernel_20848\3932318407.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp)
C:\Users\conta\AppData\Local\Temp\ipykernel_20848\3932318407.py:10: FutureWarning: The frame

In [3]:
df = df.set_index(df['id'])
df = df.drop_duplicates(subset=['id'])

d:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [4]:
for i in list(df.index):
    if type(df['genre_ids'][i]) == list:
        x = df.loc[i]
        genres = x['genre_ids']
        df.drop(axis = 0, index = i,inplace= True)
        for genre in genres:
            x['genre_ids'] = genre
            df = df.append(x)
    else: pass
df.to_excel('moviestmdb.xlsx')

C:\Users\conta\AppData\Local\Temp\ipykernel_20848\2499428502.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['genre_ids'] = genre
C:\Users\conta\AppData\Local\Temp\ipykernel_20848\2499428502.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x)
C:\Users\conta\AppData\Local\Temp\ipykernel_20848\2499428502.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x)
C:\Users\conta\AppData\Local\Temp\ipykernel_20848\2499428502.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x)
C:\Users\conta\Ap

In [5]:
url = "https://api.themoviedb.org/3/genre/movie/list?language=en-US&api_key=552dbe069b589a399d500b9bdfcbf6e7"
response = requests.request("GET", url, headers={}, data={})
jdata = json.loads(response.text)
df_genres = pd.DataFrame(jdata['genres'])
df_genres.to_excel('genres.xlsx')


In [6]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
chromedriver_autoinstaller.install()

'C:\\Users\\conta\\AppData\\Roaming\\Python\\Python39\\site-packages\\chromedriver_autoinstaller\\111\\chromedriver.exe'

In [7]:
driver = webdriver.Chrome()
driver.get('https://www.rottentomatoes.com/')


df_tom = pd.DataFrame(columns = 'tomatoscore, audience_score, lengh, original_title'.split(', '))
df_tom['original_title'] = df['original_title'].unique()
df_tom.set_index('original_title', inplace = True)
for movie in list(df_tom.index):
    try:
        search_bar =  driver.find_element(By.XPATH  ,'/html/body/div[3]/rt-header/search-algolia/search-algolia-controls/input')
        search_bar.send_keys(movie)
        search_bar.send_keys(Keys.ENTER)
        time.sleep(2)
        menu = driver.find_elements(By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "searchNav__filter", " " ))]')

        for element in menu:
            if(element.get_attribute("data-filter")=='movie'):
                menu_movie = element
                break
        menu_movie.click()
        time.sleep(1)
        driver.find_element(By.PARTIAL_LINK_TEXT,f'{movie}').click()
        time.sleep(2)
        tomatoscore =  driver.find_element(By.XPATH,'/html/body/div[3]/main/div[1]/section/div[2]/div[1]/div[1]/score-board').get_attribute('tomatometerscore')
        audience_score =  driver.find_element(By.XPATH,'/html/body/div[3]/main/div[1]/section/div[2]/div[1]/div[1]/score-board').get_attribute('audiencescore')
        lengh = driver.find_element(By.XPATH,'/html/body/div[3]/main/div[1]/section/div[2]/div[1]/div[1]/score-board/p')
        lengh = lengh.text.split(', ')[2].replace('h ','.').replace('m','').split('.')
        lengh = int(lengh[0])*60 + int(lengh[1])
        df_tom.loc[movie] = [int(tomatoscore),int(audience_score),lengh]
    except: pass








In [12]:
df_tom.tomatoscore.isna().sum()

91

In [8]:
df_tom.to_excel('rotten_tomatoes.xlsx')